In [11]:
import pandas as pd
import json

df1 = pd.read_csv('hotel_bookings.csv')

In [13]:
df1['children'] = df1['children'].fillna(0)
df1['babies'] = df1['babies'].fillna(0)
df1['country'] = df1['country'].fillna('Unknown')
df1['agent'] = df1['agent'].fillna(0)
df1['company'] = df1['company'].fillna(0)

In [15]:
df1['reservation_status_date'] = pd.to_datetime(df1['reservation_status_date'], format='%d-%m-%y', errors='coerce')
df1['arrival_date'] = pd.to_datetime(
    df1['arrival_date_year'].astype(str) + '-' + 
    df1['arrival_date_month'] + '-' + 
    df1['arrival_date_day_of_month'].astype(str),
    errors='coerce'
)

In [17]:
df1 = df1.drop(['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month'], axis=1)

df1.to_csv('cleaned_hotel_bookings.csv', index=False)

In [19]:
df = pd.read_csv('cleaned_hotel_bookings.csv')
df['arrival_date'] = pd.to_datetime(df['arrival_date'])
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])

# 1. Revenue Trends (monthly)
df['total_stay'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']
df['revenue'] = df['adr'] * df['total_stay'] * (1 - df['is_canceled'])
df['arrival_month_year'] = df['arrival_date'].dt.to_period('M')
revenue_trends = df.groupby('arrival_month_year')['revenue'].sum().to_dict()

In [21]:
# 2. Cancellation Rate
total_bookings = len(df)
cancellations = df['is_canceled'].sum()
cancellation_rate = (cancellations / total_bookings) * 100

# 3. Geographical Distribution (top 10 countries, non-canceled)
geo_dist = df[df['is_canceled'] == 0]['country'].value_counts().head(10).to_dict()

In [23]:
# 4. Lead Time Stats (precomputed summary)
lead_time_stats = {
    'mean': df['lead_time'].mean(),
    'median': df['lead_time'].median(),
    'max': df['lead_time'].max(),
    'min': df['lead_time'].min()
}

# 5. Non-Mathematical Insight: Canceled Bookings by Date
canceled_by_date = df[df['is_canceled'] == 1].groupby('reservation_status_date')['hotel'].count().to_dict()

In [25]:
analytics_data = {
    'revenue_trends': {str(k): v for k, v in revenue_trends.items()},  
    'cancellation_rate': cancellation_rate,
    'geo_distribution': geo_dist,
    'lead_time_stats': lead_time_stats,
    'canceled_by_date': {str(k): v for k, v in canceled_by_date.items()}  
}

with open('precomputed_analytics.json', 'w') as f:
    json.dump(analytics_data, f, indent=4)

print("Precomputed analytics saved to 'precomputed_analytics.json'")

Precomputed analytics saved to 'precomputed_analytics.json'
